# Load Data

In [36]:
from utils import get_data
n_clients = 3
X, y, X_test, y_test = get_data(n_clients=n_clients)

# Using Paillier cryptosystem

In [37]:
import phe as paillier
key_length=1024
pubkey, privkey = paillier.generate_paillier_keypair(n_length=key_length)

In [38]:
# define encrypt and decrypt function for this cryptosystem
def encrypt(pubkey, value):
    return pubkey.encrypt(value)

def decrypt(privkey, value):
    return privkey.decrypt(value)

## Creating server

In [39]:
from src.server import Server

In [40]:
server = Server(privkey, pubkey, decrypt)

## Creating Models

### MLP

In [41]:
from src.model import get_model_mlp

In [42]:
nb_input = 10
default_mlp = get_model_mlp(10, lr = 0.01)


## Creating clients

In [43]:
from src.client import Client
import copy

In [34]:
clients = {}
for i in range(3):
    id = i+1
    data = X[i]
    target = y[i]
    model = copy.deepcopy(default_mlp)
    client = Client(id, pubkey, model, data, target, encrypt)
    clients[id] = client

In [ ]:
# Federated Learning with encryption
    for i in range(n_iter):
        if i % 50 == 0:
            print(i)
        # Compute gradients, encrypt and aggregate
        encrypt_aggr = None
        for i in range(n_clients):
            encrypt_aggr = clients[i].encrypted_gradient(sum_to=encrypt_aggr)
        # Send aggregate to server and decrypt it
        aggr = None
        try:
            aggr = server.decrypt_list_of_aggregate(encrypt_aggr, n_clients)
        except Exception:
            aggr = server.decrypt_aggregate(encrypt_aggr, n_clients)

        # Take gradient steps
        for c in clients:
            c.gradient_step(aggr)

    return client